Datasets are as follows:

1) Dataset 1 (classification) - download the RAR file from https://archive.ics.uci.edu/ml/machine-learning-databases/00470/ with detailed information here: https://archive-beta.ics.uci.edu/ml/datasets/parkinson+s+disease+classification

2) Dataset 2 (classification) - download the XLS file from https://archive.ics.uci.edu/ml/machine-learning-databases/00350/ with detailed information here: https://archive-beta.ics.uci.edu/ml/datasets/default+of+credit+card+clients

3) Dataset 3 (classification) - download the XLS file from https://archive.ics.uci.edu/ml/machine-learning-databases/00399/ with detailed information here: https://archive-beta.ics.uci.edu/ml/datasets/meu+mobile+ksd

4) Dataset 4 (classification) - download the XLS file from https://archive.ics.uci.edu/ml/machine-learning-databases/00342/ with detailed information here: https://archive-beta.ics.uci.edu/ml/datasets/mice+protein+expression

5) Dataset 5 (regression) - download the train excel/csv file from https://archive.ics.uci.edu/ml/machine-learning-databases/00464/ with detailed information here: https://archive-beta.ics.uci.edu/ml/datasets/superconductivty+data

The following DR and FS methods to be used:

1) LDA (https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)

2) PCA (https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

3) t-SNE (https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)

4) L1 based feature selection - use the code from this blog and adjust: https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499

5) Feature Selection with Random Forest - use the code from this blog and adjust: https://towardsdatascience.com/feature-selection-using-random-forest-26d7b747597f

6) Feature Selection with XGBoost - use the code from this blog and adjust: https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/

7) Recursive Feature Elimination - use the code from this blog and adjust: https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15 and https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

Task:
Compare the classification and regression performance over the 7 different methods above. You can select any classification metrics you want, but its better to take 3-5 metrics in all (eg precsion, recall, accuracy, AUC etc.). For regression, take RMSE, MSE etc. or any other you want.Â 

For the algorithms, consider XGBoost, RF, SVM and LR for classification and MLR and Voting Regression for regression.

The number of features you want to select depends on you, e.g., you can assume 10 in the first go and select 10 from all the 7 methods and then run the classification and regression algorithms and compare. Then, you can repeat with 5 and then with 15. In this way, you will be able to acquire more information, e.g., what impact does selecting more variables or less variables has on the classification and regression performance.

For the above experiments, no hyperparameter tuning is required - just assume some default values

Write a summary of all your results (tetxual summary) by answering the following questions in detail while mentioning values of the performance metrics and graphs if feasible.
- How does the classification performance compare across the 7 DR/FS methods?
- How does the regression performance compare across the 7 DR/FS methods?
- What is the effect of increasing or decreasing the total number of your desired features on the classification and regression performance?

# Imports and Helpers

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sb
import math
import warnings
import matplotlib.pyplot as plt        
get_ipython().run_line_magic('matplotlib', 'inline')

from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

#import feature selection modules
from sklearn.feature_selection import mutual_info_classif,RFE,RFECV
from sklearn.feature_selection import mutual_info_regression

from sklearn.linear_model import Lasso, Ridge

#import classification modules
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# import regression modules
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#import split methods
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler


#PCA
from sklearn.decomposition import PCA

#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#TSNE
from sklearn.manifold import TSNE

#L1
from sklearn.linear_model import Lasso, LogisticRegression



#import performance scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve 
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle




In [2]:
# CONSTANTS

CLASSIFICATION = 'Classification'
REGRESSION = 'Regression'

REG_LASSO = 'lasso'
FS_RFFS = 'rffs'
FS_REFS = 'refs'
FS_PCA = 'pca'
FS_LDA = 'lda'

In [3]:
!gdown --id "1jLOvY2smMG9UR1AoDB2JO7POQCk3ijt2" -O "dataset1.csv"
dataset1 = pd.read_csv("dataset1.csv") # pd_speech_features

!gdown --id "1Y15hapeg-OfK_rH75VLx9j9tHIq-VT6R" -O "dataset2.csv"
dataset2 = pd.read_csv("dataset2.csv") # defaulters (credit card)

!gdown --id "1yS4AvLDmeN9wF1OEw1xmcK28KGxl6V4s" -O "dataset3.csv"
dataset3 = pd.read_csv("dataset3.csv") # mobile keystroke

!gdown --id "1QaNponzzLsGTiR8I2y7At3OJAWyKkw8a" -O "dataset4.csv"
dataset4 = pd.read_csv("dataset4.csv") # mice protein

!gdown --id "1r6j7SZeqOjD7FnYV9IO_0c2YcYWDc92o" -O "dataset5.csv"
dataset5 = pd.read_csv("dataset5.csv") # superconductor

Downloading...
From: https://drive.google.com/uc?id=1jLOvY2smMG9UR1AoDB2JO7POQCk3ijt2
To: /content/dataset1.csv
100% 5.31M/5.31M [00:00<00:00, 46.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y15hapeg-OfK_rH75VLx9j9tHIq-VT6R
To: /content/dataset2.csv
100% 2.90M/2.90M [00:00<00:00, 92.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yS4AvLDmeN9wF1OEw1xmcK28KGxl6V4s
To: /content/dataset3.csv
100% 1.51M/1.51M [00:00<00:00, 46.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QaNponzzLsGTiR8I2y7At3OJAWyKkw8a
To: /content/dataset4.csv
100% 992k/992k [00:00<00:00, 63.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1r6j7SZeqOjD7FnYV9IO_0c2YcYWDc92o
To: /content/dataset5.csv
100% 17.6M/17.6M [00:00<00:00, 64.7MB/s]


In [4]:
label_dataset1 = 'class'
label_dataset2 = 'default_payment_next_month'
label_dataset3 = 'UD_l.Enter'
label_dataset4 = 'Behavior'
label_dataset5 = 'number_of_elements'

print('-----d1-----')
print(dataset1.shape)
dataset1.dropna(inplace=True)
print(dataset1.shape)

print('-----d2-----')
print(dataset2.shape)
dataset2.dropna(inplace=True)
print(dataset2.shape)

print('-----d3-----')
print(dataset3.shape)
dataset3.dropna(inplace=True)
dataset3["AvH"] = pd.to_numeric(dataset3["AvH"])
dataset3["AvP"] = pd.to_numeric(dataset3["AvP"])
dataset3["AvA"] = pd.to_numeric(dataset3["AvA"])
dataset3.dropna(inplace=True)
print(dataset3.shape)

print('-----d4-----')
print(dataset4.shape)
dataset4.dropna(inplace=True)
dataset4.drop(columns=['MouseID'], inplace=True)
print(dataset4.shape)

print('-----d5------')
print(dataset5.shape)
dataset5.dropna(inplace=True)
print(dataset5.shape)
print('------------')

-----d1-----
(756, 755)
(756, 755)
-----d2-----
(30000, 25)
(30000, 25)
-----d3-----
(2911, 72)
(2856, 72)
-----d4-----
(1080, 82)
(1080, 81)
-----d5------
(21263, 82)
(21263, 82)
------------


# Classification Algorithms

In [5]:
# Classification Algorithms

def LogReg(trainX, testX, trainY, testY, verbose=True, clf=None, multi=False):
    if not clf:
        clf  = LogisticRegression()
    clf.fit(trainX , trainY)
    return validationmetrics(clf,testX,testY,verbose=verbose,multi=multi)

def SVM(trainX, testX, trainY, testY, svmtype="SVC", verbose=True, clf=None, multi=False):
    # for one vs all
    if not clf:
        if svmtype == "Linear":
            clf = svm.LinearSVC()
        else:
            clf = svm.SVC()
    clf.fit(trainX , trainY)
    return validationmetrics(clf,testX,testY,verbose=verbose,multi=multi)


def RandomForest(trainX, testX, trainY, testY, verbose=True, clf=None, multi=False):
    if not clf:
        clf  = RandomForestClassifier()
    clf.fit(trainX , trainY)
    return validationmetrics(clf,testX,testY,verbose=verbose,multi=multi)


def XgBoost(trainX, testX, trainY, testY, verbose=True, clf=None, multi=False):
    if not clf:
      objective = 'binary:logistic'
      if multi == True:
        objective = 'multi:softmax'
      clf = XGBClassifier(random_state=1,learning_rate=0.01, silent=1, objective=objective)
    clf.fit(trainX,trainY)
    return validationmetrics(clf,testX,testY,verbose=verbose,multi=multi)

def RunClassificationAlgorithms(trainX, testX, trainY, testY, verbose=True, multi=False):
  print("Running Classification Algorithms")

  print("Running XgBoost")
  XgBoost(trainX, testX, trainY, testY, verbose=verbose, multi=multi)
  print('\n')
  print("Running LR")
  LogReg(trainX, testX, trainY, testY, verbose=verbose, multi=multi)
  print('\n')
  print("Running SVM")
  SVM(trainX, testX, trainY, testY, verbose=verbose, multi=multi)
  print('\n')
  print("Running Random Forest")
  RandomForest(trainX, testX, trainY, testY, verbose=verbose, multi=multi)
  print('\n')


# Regression Algorithms

In [6]:
# Regression Algorithms
def LinearReg(trainX, testX, trainY, testY, verbose=True, clf=None):
    if not clf:
        clf  = LinearRegression()
    clf.fit(trainX , trainY)
    return validationmetrics_reg(clf, testX, testY, verbose=verbose)

def RandomForestReg(trainX, testX, trainY, testY, verbose=True, clf=None):
    if not clf:
        clf = RandomForestRegressor(n_estimators=100)
    clf.fit(trainX , trainY)
    return validationmetrics_reg(clf, testX, testY, verbose=verbose)

def PolynomialReg(trainX, testX, trainY, testY, degree=3, verbose=True, clf=None):
    poly = PolynomialFeatures(degree = degree)
    X_poly = poly.fit_transform(trainX)
    poly.fit(X_poly, trainY)
    if not clf:
        clf = LinearRegression() 
    clf.fit(X_poly, trainY)
    return validationmetrics_reg(clf, poly.fit_transform(testX), testY, verbose=verbose)

def SupportVectorRegression(trainX, testX, trainY, testY, verbose=True, clf=None):
    if not clf:
        clf = SVR(kernel="rbf")
    clf.fit(trainX , trainY)
    return validationmetrics_reg(clf, testX, testY, verbose=verbose)

def DecisionTreeReg(trainX, testX, trainY, testY, verbose=True, clf=None):
    if not clf:
        clf = DecisionTreeRegressor()
    clf.fit(trainX , trainY)
    return validationmetrics_reg(clf, testX, testY, verbose=verbose)

def GradientBoostingReg(trainX, testX, trainY, testY, verbose=True, clf=None):
    if not clf:
        clf = GradientBoostingRegressor()
    clf.fit(trainX , trainY)
    return validationmetrics_reg(clf, testX, testY, verbose=verbose)

def AdaBooostReg(trainX, testX, trainY, testY, verbose=True, clf=None):
    if not clf:
        clf = AdaBoostRegressor(random_state=0, n_estimators=100)
    clf.fit(trainX , trainY)
    return validationmetrics_reg(clf, testX, testY, verbose=verbose)

def VotingReg(trainX, testX, trainY, testY, verbose=True, clf=None):
    rf = RandomForestRegressor(n_estimators=100)
    sv = svm.SVR(kernel="rbf")
    ada = AdaBoostRegressor(random_state=0, n_estimators=100)
    gb = GradientBoostingRegressor()
    dt = DecisionTreeRegressor()

    if not clf:
        clf = VotingRegressor([('rf', rf), ('sv', sv), ('ada', ada), ('gb', gb), ('dt', dt)])
    clf.fit(trainX , trainY)
    return validationmetrics_reg(clf, testX, testY, verbose=verbose)

def RunRegressionAlgorithms(trainX, testX, trainY, testY):
  print("Running Regression Algorithms")

  print("Running Linear Regression")
  LinearReg(trainX, testX, trainY, testY)
  print('\n')
  
  # print("Running Polynomial Regression")
  # PolynomialReg(trainX, testX, trainY, testY, verbose=verbose)

  print("Running Voting Regressor")
  VotingReg(trainX, testX, trainY, testY)
  print('\n')


# Validation metrics


In [7]:
# Validation metrics for classification

def validationmetrics(model, testX, testY, verbose=True, multi=False):   
    predictions = model.predict(testX)
    
    if model.__class__.__module__.startswith('sklearn.linear_model._coordinate_descent') or model.__class__.__module__.startswith('sklearn.linear_model._ridge') or model.__class__.__module__.startswith('lightgbm'):

      for i in range(0, predictions.shape[0]):
        predictions[i] = 1 if predictions[i] >= 0.5 else 0

    average = 'binary'
    if multi == True:
      average='weighted'

    accuracy = accuracy_score(testY, predictions) * 100
    precision = precision_score(testY, predictions, pos_label=1, labels=[0,1], average=average) * 100
    recall = recall_score(testY, predictions,pos_label=1,labels=[0,1], average=average) * 100
    if not multi:
      fpr , tpr, _ = roc_curve(testY, predictions)
      auc_val = auc(fpr, tpr)
    f_score = f1_score(testY, predictions, average=average)

    if verbose:
        print("Prediction Vector: \n", predictions)
        print("\n Accuracy: \n", accuracy)
        print("\n Precision of event Happening: \n", precision)
        print("\n Recall of event Happening: \n", recall)
        if not multi:
          print("\n AUC: \n",auc_val)
          print("\n Confusion Matrix: \n", confusion_matrix(testY, predictions,labels=[0,1]))
        print("\n F-Score:\n", f_score)
        #confusion Matrix
    
    res_map = {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f_score": f_score,
      "model_obj": model
    }

    if not multi:
      res_map["auc_val"]: auc_val
    
    return res_map


In [8]:
#Validation metrics for Regression algorithms

def validationmetrics_reg(model, testX, testY, verbose=True):
    predictions = model.predict(testX)
    
    r2 = r2_score(testY,predictions)
    r2_adjusted = 1-(1-r2)*(testX.shape[0]-1)/(testX.shape[0]-testX.shape[1]-1)
    mse = mean_squared_error(testY,predictions)
    rmse = math.sqrt(mse)
    
    if verbose:
      print("R-Squared Value: ", r2)
      print("Adjusted R-Squared: ", r2_adjusted)
      print("RMSE: ", rmse)
    
    res_map = {
      "r2": r2,
      "r2_adjusted": r2_adjusted,
      "rmse": rmse,
      "model_obj": model
    }

    return res_map


# Dimensionality Reduction & Feature Selection

In [9]:
#Train Test Split: splitting manually
def traintestsplit(df, label_col, split=None, random=None):
  #make a copy of the label column and store in y
  df_cpy = df.copy()
  y = df_cpy[label_col].copy()

  #now delete the original
  X = df_cpy.drop(label_col,axis=1)

  if split == None:
    return X, y

  #manual split
  trainX, testX, trainY, testY = train_test_split(X, y, test_size=split, random_state=random)
  return X, y, trainX, testX, trainY, testY

#helper function which only splits into X and y
def XYsplit(df, label_col):
    df_cpy = df.copy()
    y = df_cpy[label_col].copy()
    X = df_cpy.drop(label_col,axis=1)
    return X,y


##LDA

In [10]:
def LDAImpl(df, label_col, ncomponents, multi=False):    
    df_cpy = df.copy()
    X,y = XYsplit(df_cpy, label_col)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    sc = StandardScaler()
    X_scaled = sc.fit_transform(X_train)

    lda = LinearDiscriminantAnalysis(n_components=ncomponents)
    X_lda = lda.fit_transform(X_scaled, y_train)
    X_lda = pd.DataFrame(X_lda)
    X_train_lda, X_test_lda, y_train, y_test = train_test_split(X_lda, y_train, test_size=0.20, shuffle=True, random_state=2)

    RunClassificationAlgorithms(X_train_lda, X_test_lda, y_train, y_test, verbose=True, multi=multi)
    print('\n\n')
    RunRegressionAlgorithms(X_train_lda, X_test_lda, y_train, y_test)

###Running LDA with 5 features

In [11]:
print('Running on dataset1')
LDAImpl(dataset1, label_dataset1, ncomponents=5, multi=False)
print('-----')
print('Running on dataset2')
LDAImpl(dataset2, label_dataset2, ncomponents=5, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
LDAImpl(dataset3, label_dataset3, ncomponents=5, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
LDAImpl(dataset4, label_dataset4, ncomponents=5, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
LDAImpl(dataset5, label_dataset5, ncomponents=5, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')
print('-----')


Running on dataset1


/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(754, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 0 0 0 1 1 1 1]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 AUC: 
 1.0

 Confusion Matrix: 
 [[24  0]
 [ 0 97]]

 F-Score:
 1.0


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 0 0 0 1 1 1 1]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 AUC: 
 1.0

 Confusion Matrix: 
 [[24  0]
 [ 0 97]]

 F-Score:
 1.0


Running SVM
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(24, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.5625

 Precision of event Happening: 
 65.27545909849749

 Recall of event Happening: 
 36.61048689138577

 AUC: 
 0.6551853390657177

 Confusion Matrix: 
 [[3524  208]
 [ 677  391]]

 F-Score:
 0.46910617876424715


Running LR
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.47916666666667

 Precision of event Happening: 
 72.65822784810126

 Recall of event Happening: 
 26.872659176029963

 AUC: 
 0.619893842503944

 Confusion Matrix: 
 [[3624  108]
 [ 781  287]]

 F-Score:
 0.39234449760765555


Running SVM
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.52083333333333

 Precision of event Happening: 
 63.60902255639098

 Recall of event Happening: 
 39.60674157303371

 AUC: 
 0.6656114141888556

 Confusion Matrix: 
 [[3490  242]
 [ 645  423]]

 F-Score:
 0.48817080207732266


Running Random Forest
Prediction Vector: 
 [0 1 0 ... 0 0 0]

 Accuracy: 
 72.25

 Preci

...

Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0


Running LR


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true sampl

Prediction Vector: 
 [2 3 7 ... 5 6 6]

 Accuracy: 
 91.26984126984127

 Precision of event Happening: 
 96.07843137254902

 Recall of event Happening: 
 100.0

 F-Score:
 0.911348089070941


Running SVM
Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9875876039582869
Adjusted R-Squared:  0.9875693289346684
RMSE:  0.16001253164604926


Running Voting Regressor
R-Squared Value:  0.9998988482530338
Adjusted R-Squared:  0.9998986993252555
RMSE:  0.01444484474275176


-----
-----
-----
-----
-----


###Running LDA with 10 features

In [12]:
print('Running on dataset1')
LDAImpl(dataset1, label_dataset1, ncomponents=10, multi=False)
print('-----')
print('Running on dataset2')
LDAImpl(dataset2, label_dataset2, ncomponents=10, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
LDAImpl(dataset3, label_dataset3, ncomponents=10, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
LDAImpl(dataset4, label_dataset4, ncomponents=10, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
LDAImpl(dataset5, label_dataset5, ncomponents=10, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')
print('-----')


Running on dataset1


/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(754, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 0 0 0 1 1 1 1]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 AUC: 
 1.0

 Confusion Matrix: 
 [[24  0]
 [ 0 97]]

 F-Score:
 1.0


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 0 0 0 1 1 1 1]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 AUC: 
 1.0

 Confusion Matrix: 
 [[24  0]
 [ 0 97]]

 F-Score:
 1.0


Running SVM
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(24, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.5625

 Precision of event Happening: 
 65.27545909849749

 Recall of event Happening: 
 36.61048689138577

 AUC: 
 0.6551853390657177

 Confusion Matrix: 
 [[3524  208]
 [ 677  391]]

 F-Score:
 0.46910617876424715


Running LR
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.47916666666667

 Precision of event Happening: 
 72.65822784810126

 Recall of event Happening: 
 26.872659176029963

 AUC: 
 0.619893842503944

 Confusion Matrix: 
 [[3624  108]
 [ 781  287]]

 F-Score:
 0.39234449760765555


Running SVM
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.52083333333333

 Precision of event Happening: 
 63.60902255639098

 Recall of event Happening: 
 39.60674157303371

 AUC: 
 0.6656114141888556

 Confusion Matrix: 
 [[3490  242]
 [ 645  423]]

 F-Score:
 0.48817080207732266


Running Random Forest
Prediction Vector: 
 [0 1 0 ... 0 0 0]

 Accuracy: 
 72.2708333333

...

Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0


Running LR


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true sampl

Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 91.005291005291

 Precision of event Happening: 
 94.23076923076923

 Recall of event Happening: 
 100.0

 F-Score:
 0.9095812061960508


Running SVM
Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 99.97060552616108

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 0.9997008295773727





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9877765644354293
Adjusted R-Squared:  0.9877477440745344
RMSE:  0.15878988279560946


Running Voting Regressor
R-Squared Value:  0.9999004482124905
Adjusted R-Squared:  0.9999002134897377
RMSE:  0.014330149315410873


-----
-----
-----
-----
-----


###Running LDA with 15 features

In [13]:
print('Running on dataset1')
LDAImpl(dataset1, label_dataset1, ncomponents=15, multi=False)
print('-----')
print('Running on dataset2')
LDAImpl(dataset2, label_dataset2, ncomponents=15, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
LDAImpl(dataset3, label_dataset3, ncomponents=15, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
LDAImpl(dataset4, label_dataset4, ncomponents=15, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
LDAImpl(dataset5, label_dataset5, ncomponents=15, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')
print('-----')


Running on dataset1


/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(754, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 0 0 0 1 1 1 1]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 AUC: 
 1.0

 Confusion Matrix: 
 [[24  0]
 [ 0 97]]

 F-Score:
 1.0


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 0
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 0 0 0 1 1 1 1]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 AUC: 
 1.0

 Confusion Matrix: 
 [[24  0]
 [ 0 97]]

 F-Score:
 1.0


Running SVM
Prediction Vector: 
 [1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(24, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.5625

 Precision of event Happening: 
 65.27545909849749

 Recall of event Happening: 
 36.61048689138577

 AUC: 
 0.6551853390657177

 Confusion Matrix: 
 [[3524  208]
 [ 677  391]]

 F-Score:
 0.46910617876424715


Running LR
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.47916666666667

 Precision of event Happening: 
 72.65822784810126

 Recall of event Happening: 
 26.872659176029963

 AUC: 
 0.619893842503944

 Confusion Matrix: 
 [[3624  108]
 [ 781  287]]

 F-Score:
 0.39234449760765555


Running SVM
Prediction Vector: 
 [0 0 0 ... 0 0 0]

 Accuracy: 
 81.52083333333333

 Precision of event Happening: 
 63.60902255639098

 Recall of event Happening: 
 39.60674157303371

 AUC: 
 0.6656114141888556

 Confusion Matrix: 
 [[3490  242]
 [ 645  423]]

 F-Score:
 0.48817080207732266


Running Random Forest
Prediction Vector: 
 [0 1 0 ... 0 0 0]

 Accuracy: 
 72.2083333333

...

Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0


Running LR


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true sampl

Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 91.005291005291

 Precision of event Happening: 
 94.23076923076923

 Recall of event Happening: 
 100.0

 F-Score:
 0.9095812061960508


Running SVM
Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 99.94121105232216

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 0.9993902164092265





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9877765644354293
Adjusted R-Squared:  0.9877477440745344
RMSE:  0.15878988279560946


Running Voting Regressor
R-Squared Value:  0.9999004482124905
Adjusted R-Squared:  0.9999002134897377
RMSE:  0.014330149315410873


-----
-----
-----
-----
-----


##PCA

In [14]:
# PCA
def PCAImpl(df, label_col, ncomponents, multi=False):    
    df_cpy = df.copy()
    X,y = XYsplit(df_cpy, label_col)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    sc = StandardScaler()
    X_scaled = sc.fit_transform(X_train)

    pca = PCA(n_components=6)
    X_pca = pca.fit_transform(X_scaled)
    X_pca = pd.DataFrame(X_pca)
    X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y_train, test_size=0.20, shuffle=True, random_state=2)

    RunClassificationAlgorithms(X_train_pca, X_test_pca, y_train, y_test, verbose=True, multi=multi)
    print('\n\n')
    RunRegressionAlgorithms(X_train_pca, X_test_pca, y_train, y_test)

###Running PCA with 5 features

In [15]:
print('Running on dataset1')
PCAImpl(dataset1, label_dataset1, ncomponents=5, multi=False)
print('-----')
print('Running on dataset2')
PCAImpl(dataset2, label_dataset2, ncomponents=5, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
PCAImpl(dataset3, label_dataset3, ncomponents=5, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
PCAImpl(dataset4, label_dataset4, ncomponents=5, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
PCAImpl(dataset5, label_dataset5, ncomponents=5, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1
Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0
 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1
 1 1 1 1 1 1 1 1 1 1]

 Accuracy: 
 80.99173553719008

 Precision of event Happening: 
 87.0

 Recall of event Happening: 
 89.69072164948454

 AUC: 
 0.6776202749140894

 Confusion Matrix: 
 [[11 13]
 [10 87]]

 F-Score:
 0.883248730964467


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1]

 Accuracy: 
 82.64462809917356

 Precision of event Happening: 
 88.0

 Recall of event Happening: 
 90.72164948453609

 AUC: 
 0.7036082474226804

 Confusion Ma

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [456. 462. 359. 512. 346. 464. 437. 410. 505. 456. 389. 503. 467. 530.
 448. 380. 503. 462. 566. 459. 480. 462. 389. 434. 462. 462. 462. 344.
 520. 350. 480. 354. 332. 530. 522. 405. 332. 350. 359. 410. 359. 405.
 462. 345. 434. 434. 405. 389. 384. 328. 467. 389. 433. 335. 405. 464.
 520. 380. 389. 462. 369. 522. 501. 566. 354. 354. 456. 464. 359. 368.
 600. 505. 433. 405. 345. 369. 399. 467. 503. 389. 344. 512. 467. 404.
 462. 389. 410. 481. 422. 464. 359. 405. 433. 433. 505. 354. 456. 405.
 423. 433. 478. 388. 512. 410. 462. 332. 433. 342. 522. 480. 359. 503.
 433. 491. 410. 464. 359. 472. 421. 366. 462. 503. 369. 428. 428. 368.
 464. 384. 504. 554. 593. 332. 380. 399. 503. 456. 433. 516. 424. 332.
 437. 369. 457. 505. 472. 423. 369. 410. 443. 505. 503. 350. 384. 462.
 368. 389. 449. 520. 405. 279. 404. 374. 328. 541. 434. 462. 377. 472.
 328. 359. 450. 503. 462. 503. 434. 410. 512. 410. 505. 443. 480. 416.
 328. 501. 467. 405. 418. 464. 368. 328. 584. 462. 279. 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true sampl

Prediction Vector: 
 [ 359.  416.  424.  401.  462.  380.  796.  289.  259.  399.  389.  878.
  696. 1323. 1072.  495.  817.  410.  407.  730.  359.  621.  346.  545.
  410.  300.  600.  344.  253.  369.  359.  478.  416.  588.  520.  424.
  384.  442.  359.  371.  520. 2617. 1785.  354.  664.  371. 1112.  437.
  443.  505.  467.  384.  505.  371.  405.  417.  675.  561.  389.  554.
  354.  472.  481.  478.  462.  347.  407.  255.  359.  259.  524.  505.
  505.  459.  354.  442.  424.  437.  733.  384.  347.  382.  520.  354.
  472.  384.  443.  553.  389.  350.  681. 1057.  664.  433.  542.  462.
  530.  467.  467.  440.  371.  733.  389.  462.  410.  500.  345.  462.
  426.  531.  535.  530. 1756.  733.  354.  366.  359.  407.  399.  300.
  462.  293.  410.  371.  440.  259.  380.  344.  545.  654.  359.  384.
  696.  462.  814.  459. 1376.  733.  274.  389.  437.  410.  374.  278.
  462.  369.  410.  467.  545.  505.  497.  531.  410.  462.  354.  367.
  354.  405.  459.  366.  259.

...

Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 90.8289241622575

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 0.9071675933400235


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 97.38389182833627

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 0.9720124616575642


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 98.94179894179894

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 97.95918367346938

 F-Score:
 0.9893224637813124





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9069877171260884
Adjusted R-Squared:  0.9068233360665174
RMSE:  0.4380224159802926


Running Voting Regressor
R-Squared Value:  0.9856462096481821
Adjusted R-Squared:  0.9856208421247326
RMSE:  0.17207171362577023


-----
-----
-----
-----


###Running PCA with 10 features

In [16]:
print('Running on dataset1')
PCAImpl(dataset1, label_dataset1, ncomponents=10, multi=False)
print('-----')
print('Running on dataset2')
PCAImpl(dataset2, label_dataset2, ncomponents=10, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
PCAImpl(dataset3, label_dataset3, ncomponents=10, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
PCAImpl(dataset4, label_dataset4, ncomponents=10, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
PCAImpl(dataset5, label_dataset5, ncomponents=10, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1
Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0
 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1
 1 1 1 1 1 1 1 1 1 1]

 Accuracy: 
 80.99173553719008

 Precision of event Happening: 
 87.0

 Recall of event Happening: 
 89.69072164948454

 AUC: 
 0.6776202749140894

 Confusion Matrix: 
 [[11 13]
 [10 87]]

 F-Score:
 0.883248730964467


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1]

 Accuracy: 
 82.64462809917356

 Precision of event Happening: 
 88.0

 Recall of event Happening: 
 90.72164948453609

 AUC: 
 0.7036082474226804

 Confusion Ma

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [456. 462. 359. 512. 346. 397. 437. 410. 428. 456. 389. 503. 467. 530.
 448. 434. 503. 462. 566. 459. 480. 462. 389. 434. 462. 462. 462. 347.
 520. 350. 480. 354. 389. 530. 522. 405. 332. 350. 359. 410. 464. 405.
 462. 345. 590. 434. 405. 389. 384. 328. 467. 389. 433. 335. 405. 464.
 520. 359. 389. 462. 369. 522. 501. 566. 354. 354. 456. 464. 359. 368.
 600. 505. 368. 405. 345. 410. 399. 467. 503. 389. 344. 512. 328. 404.
 462. 389. 410. 481. 422. 464. 359. 405. 433. 433. 505. 354. 456. 405.
 423. 433. 478. 335. 512. 410. 462. 332. 433. 342. 522. 480. 359. 503.
 433. 491. 462. 464. 359. 472. 421. 366. 462. 503. 369. 379. 428. 368.
 464. 462. 434. 554. 593. 332. 380. 399. 503. 410. 433. 516. 424. 332.
 388. 369. 457. 505. 472. 423. 369. 410. 443. 505. 503. 410. 384. 462.
 368. 389. 449. 520. 405. 279. 404. 374. 328. 541. 434. 462. 377. 472.
 328. 359. 450. 503. 462. 503. 434. 522. 512. 410. 505. 443. 480. 416.
 328. 501. 467. 405. 590. 464. 368. 328. 584. 462. 279. 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true sampl

Prediction Vector: 
 [ 359.  416.  424.  401.  462.  380.  796.  289.  259.  399.  389.  878.
  696. 1323. 1072.  495.  817.  410.  407.  730.  359.  621.  346.  545.
  410.  300.  600.  344.  253.  369.  359.  478.  416.  588.  520.  424.
  384.  442.  359.  371.  520. 2617. 1785.  354.  664.  371. 1112.  437.
  443.  505.  467.  384.  505.  371.  405.  417.  675.  561.  389.  554.
  354.  472.  481.  478.  354.  347.  407.  255.  359.  259.  524.  505.
  505.  459.  354.  369.  424.  437.  733.  384.  347.  382.  520.  354.
  472.  384.  443.  553.  389.  350.  681. 1057.  664.  433.  542.  462.
  530.  467.  467.  440.  371.  733.  389.  462.  410.  500.  345.  462.
  426.  531.  535.  530. 1756.  733.  354.  366.  359.  407.  399.  300.
  462.  293.  410.  371.  440.  259.  380.  344.  545.  654.  359.  384.
  696.  462.  814.  459. 1376.  733.  274.  389.  437.  410.  374.  278.
  462.  442.  410.  467.  545.  505.  497.  531.  410.  462.  354.  367.
  354.  405.  459.  366.  389.

...

Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 90.91710758377425

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 0.9080282348360112


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 97.38389182833627

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 0.9720124616575642


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 98.97119341563786

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 97.95918367346938

 F-Score:
 0.9896134389489288





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9069877107521289
Adjusted R-Squared:  0.9068233296812932
RMSE:  0.43802243098872484


Running Voting Regressor
R-Squared Value:  0.9857215757986935
Adjusted R-Squared:  0.9856963414702081
RMSE:  0.17161937838308605


-----
-----
-----
-----


###Running PCA with 15 features

In [17]:
print('Running on dataset1')
PCAImpl(dataset1, label_dataset1, ncomponents=15, multi=False)
print('-----')
print('Running on dataset2')
PCAImpl(dataset2, label_dataset2, ncomponents=15, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
PCAImpl(dataset3, label_dataset3, ncomponents=15, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
PCAImpl(dataset4, label_dataset4, ncomponents=15, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
PCAImpl(dataset5, label_dataset5, ncomponents=15, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1
Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0
 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1
 1 1 1 1 1 1 1 1 1 1]

 Accuracy: 
 80.99173553719008

 Precision of event Happening: 
 87.0

 Recall of event Happening: 
 89.69072164948454

 AUC: 
 0.6776202749140894

 Confusion Matrix: 
 [[11 13]
 [10 87]]

 F-Score:
 0.883248730964467


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1]

 Accuracy: 
 82.64462809917356

 Precision of event Happening: 
 88.0

 Recall of event Happening: 
 90.72164948453609

 AUC: 
 0.7036082474226804

 Confusion Ma

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [456. 462. 359. 512. 346. 359. 437. 410. 428. 520. 389. 503. 467. 530.
 448. 434. 503. 462. 566. 459. 480. 462. 332. 434. 462. 462. 462. 347.
 424. 350. 480. 354. 389. 530. 522. 405. 332. 350. 359. 410. 359. 405.
 462. 345. 590. 434. 405. 389. 384. 328. 467. 389. 433. 335. 405. 464.
 520. 593. 389. 462. 369. 522. 501. 566. 354. 344. 456. 464. 359. 368.
 600. 505. 433. 405. 345. 410. 399. 389. 503. 389. 344. 512. 328. 404.
 462. 389. 410. 481. 422. 464. 359. 405. 433. 464. 505. 354. 456. 405.
 423. 433. 478. 388. 512. 410. 462. 332. 433. 342. 522. 480. 410. 503.
 433. 491. 462. 464. 359. 472. 421. 366. 462. 503. 369. 379. 428. 368.
 464. 410. 434. 554. 593. 332. 380. 399. 503. 456. 433. 516. 424. 332.
 437. 369. 457. 505. 472. 423. 369. 410. 443. 505. 503. 410. 384. 462.
 368. 389. 449. 520. 405. 279. 404. 374. 328. 541. 434. 462. 377. 472.
 328. 380. 450. 503. 554. 503. 451. 522. 512. 410. 505. 443. 480. 426.
 326. 279. 467. 405. 590. 464. 368. 328. 584. 462. 279. 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true sampl

Prediction Vector: 
 [ 359.  416.  424.  401.  462.  380.  796.  289.  259.  399.  389.  878.
  696. 1323. 1072.  495.  817.  410.  407.  730.  359.  621.  346.  545.
  410.  300.  600.  344.  253.  369.  359.  462.  416.  588.  520.  424.
  384.  442.  359.  371.  520. 2617. 1785.  354.  664.  371. 1112.  437.
  443.  505.  467.  384.  505.  371.  405.  417.  675.  561.  389.  554.
  354.  472.  481.  478.  509.  347.  407.  255.  359.  259.  524.  505.
  505.  459.  354.  369.  424.  437.  733.  384.  347.  382.  424.  354.
  472.  384.  443.  553.  389.  350.  681. 1057.  664.  433.  542.  462.
  530.  467.  467.  440.  371.  733.  389.  462.  410.  500.  345.  462.
  426.  531.  535.  530. 1756.  733.  354.  366.  359.  407.  399.  300.
  462.  293.  410.  371.  440.  259.  380.  344.  545.  654.  359.  384.
  696.  462.  814.  459. 1376.  733.  274.  389.  437.  410.  374.  278.
  462.  442.  410.  467.  545.  505.  497.  531.  410.  462.  354.  367.
  354.  405.  459.  366.  259.

...

Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 90.97589653145208

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 0.9085852356418069


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 97.38389182833627

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 0.9720124616575642


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 98.97119341563786

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 97.95918367346938

 F-Score:
 0.9896608667912442





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9069876976121515
Adjusted R-Squared:  0.9068233165180934
RMSE:  0.4380224619287474


Running Voting Regressor
R-Squared Value:  0.9857216430956967
Adjusted R-Squared:  0.9856964088861456
RMSE:  0.17161897394477774


-----
-----
-----
-----


##TSNE

In [18]:
# t-SNE
def TSNEImpl(df, label_col, ncomponents, multi=False):
    df_cpy = df.copy()
    X,y = XYsplit(df_cpy, label_col)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    sc = StandardScaler()
    X_scaled = sc.fit_transform(X_train)

    tsne = TSNE(n_components=ncomponents)
    X_tsne = tsne.fit_transform(X_scaled)
    X_tsne = pd.DataFrame(X_tsne)
    X_train_tsne, X_test_tsne, y_train, y_test = train_test_split(X_tsne, y_train, test_size=0.20, shuffle=True, random_state=2)

    RunClassificationAlgorithms(X_train_tsne, X_test_tsne, y_train, y_test, verbose=True, multi=multi)
    print('\n\n')
    RunRegressionAlgorithms(X_train_tsne, X_test_tsne, y_train, y_test)


###Running TSNE with 3 features

In [19]:
print('Running on dataset1')
TSNEImpl(dataset1, label_dataset1, ncomponents=3, multi=False)
print('-----')
print('Running on dataset2')
TSNEImpl(dataset2, label_dataset2, ncomponents=3, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
TSNEImpl(dataset3, label_dataset3, ncomponents=3, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
TSNEImpl(dataset4, label_dataset4, ncomponents=3, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
TSNEImpl(dataset5, label_dataset5, ncomponents=3, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1
Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1]

 Accuracy: 
 83.47107438016529

 Precision of event Happening: 
 85.3211009174312

 Recall of event Happening: 
 95.87628865979381

 AUC: 
 0.6460481099656358

 Confusion Matrix: 
 [[ 8 16]
 [ 4 93]]

 F-Score:
 0.9029126213592232


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 0 1 1 1 1]

 Accuracy: 
 84.29752066115702

 Precision of event Happening: 
 86.11111111111111

 Recall of event Happening: 
 95.87628865979381

 AUC: 
 0.66688

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [482. 462. 332. 335. 371. 664. 405. 443. 487. 374. 482. 664. 488. 481.
 520. 505. 384. 501. 332. 332. 350. 481. 347. 316. 332. 405. 451. 384.
 520. 448. 482. 405. 664. 481. 448. 332. 350. 588. 316. 467. 448. 359.
 351. 451. 332. 482. 359. 384. 482. 428. 404. 384. 433. 371. 405. 424.
 424. 332. 347. 566. 374. 462. 407. 416. 405. 437. 487. 377. 448. 467.
 588. 520. 433. 467. 467. 423. 332. 384. 664. 482. 371. 462. 350. 467.
 482. 384. 482. 410. 405. 357. 495. 664. 664. 433. 664. 351. 481. 332.
 420. 405. 335. 664. 437. 462. 342. 481. 416. 342. 581. 410. 380. 481.
 433. 505. 581. 487. 369. 593. 359. 369. 371. 259. 359. 335. 451. 487.
 377. 462. 342. 664. 488. 374. 664. 410. 351. 467. 433. 505. 664. 259.
 404. 462. 448. 259. 405. 350. 407. 332. 326. 478. 405. 423. 369. 388.
 332. 467. 332. 451. 450. 424. 467. 733. 405. 664. 350. 416. 664. 487.
 520. 424. 462. 481. 374. 505. 404. 481. 371. 423. 377. 581. 359. 467.
 410. 482. 384. 593. 488. 359. 359. 531. 346. 482. 664. 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true sampl

Prediction Vector: 
 [ 531.  562.  505.  401.  382.  380.  405.  443.  397.  467.  462.  681.
  578.  545.  664.  505.  531.  501.  467.  450.  467.  724.  401.  501.
  467.  562.  401.  423.  690.  359.  480.  382.  562.  588.  708.  664.
  357.  439.  503.  383.  721.  733. 1182.  401.  505.  562.  733.  346.
  569.  359.  451.  437.  433.  462.  405.  380.  654.  542.  401.  600.
  501.  660.  531.  259.  451.  383.  637.  380. 1215.  401.  721.  733.
  433.  376.  410.  359.  505.  437.  975.  569.  462.  562.  359.  383.
  660.  451.  443.  531.  405.  359.  733.  491.  654.  433.  380.  354.
  545.  450.  481.  653.  462.  593.  437.  462.  501.  481.  416.  490.
  535.  443.  481.  690.  433.  733.  604.  377.  424. 1182.  447.  300.
  613.  259.  424.  462.  401.  240.  505.  405.  708.  593.  359.  569.
  681.  462.  654.  450.  433.  733.  274.  397.  437.  501.  359.  433.
  382.  325.  359.  410.  604.  733. 1182.  359.  359.  613.  437.  705.
  467.  401.  450.  462.  383.

...

Prediction Vector: 
 [4 2 5 ... 5 7 4]

 Accuracy: 
 36.6549088771311

 Precision of event Happening: 
 89.28571428571429

 Recall of event Happening: 
 51.02040816326531

 F-Score:
 0.3558358047841033


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 6 6 6]

 Accuracy: 
 82.06937095825985

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 95.91836734693877

 F-Score:
 0.8182476631539937


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [2 3 6 ... 5 6 6]

 Accuracy: 
 98.91240446796003

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 97.95918367346938

 F-Score:
 0.9890968961149026





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.26287471224468784
Adjusted R-Squared:  0.2622239247628556
RMSE:  1.2330958597539088


Running Voting Regressor
R-Squared Value:  0.9068182376235969
Adjusted R-Squared:  0.9067359700287972
RMSE:  0.4384212989717953


-----
-----
-----
-----


##XGBoost

In [20]:
#XGBoost Classifier

def XGBImpl(df, label_col, multi=False):
  df_cpy = df.copy()
  X,y = XYsplit(df_cpy, label_col)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

  # First we need to normalize the data
  sc = StandardScaler()
  sc.fit_transform(X_train)
  sc.transform(X_test)

  scaler = StandardScaler()
  scaler.fit(X_train.fillna(0))

  # Now perform L1

  model = XGBClassifier()
  model.fit(X_train, y_train)

  # make predictions for test set
  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]

  accuracy = accuracy_score(y_test, predictions)
  feature_important = model.get_booster().get_score(importance_type='weight')
  selected_feat = list(feature_important.keys())
  
  ndf = df_cpy[df_cpy.columns.intersection(selected_feat)]
  ndf[label_col] = df_cpy[label_col]

  X,y = XYsplit(ndf.copy(), label_col)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

  RunClassificationAlgorithms(X_train, X_test, y_train, y_test, verbose=True, multi=multi)
  print('\n\n')
  RunRegressionAlgorithms(X_train, X_test, y_train, y_test)

In [21]:
print('Running on dataset1')
XGBImpl(dataset1, label_dataset1, multi=False)
print('-----')
print('Running on dataset2')
XGBImpl(dataset2, label_dataset2, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
XGBImpl(dataset3, label_dataset3, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
XGBImpl(dataset4, label_dataset4, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
XGBImpl(dataset5, label_dataset5, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 0 1]

 Accuracy: 
 83.55263157894737

 Precision of event Happening: 
 86.1788617886179

 Recall of event Happening: 
 92.98245614035088

 AUC: 
 0.7412280701754386

 Confusion Matrix: 
 [[ 21  17]
 [  8 106]]

 F-Score:
 0.8945147679324894


Running LR
Prediction Vector: 
 [0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 80.26315789473685

 Preci

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



 Accuracy: 
 75.0

 Precision of event Happening: 
 75.0

 Recall of event Happening: 
 100.0

 AUC: 
 0.5

 Confusion Matrix: 
 [[  0  38]
 [  0 114]]

 F-Score:
 0.8571428571428571


Running Random Forest
Prediction Vector: 
 [0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 86.18421052631578

 Precision of event Happening: 
 87.2

 Recall of event Happening: 
 95.6140350877193

 AUC: 
 0.7675438596491229

 Confusion Matrix: 
 [[ 22  16]
 [  5 109]]

 F-Score:
 0.9121338912133891





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.19091534446076996
Adjusted R-Squared:  3.002816114531537
RMSE:  0.3894911718044526


Running Voting Regressor
R-Squared Value:  0.49031034550326147
Adjusted 

...

Prediction Vector: 
 [6 5 4 ... 2 4 4]

 Accuracy: 
 62.896778744415705

 Precision of event Happening: 
 94.64285714285714

 Recall of event Happening: 
 86.88524590163934

 F-Score:
 0.6008483586605413


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [6 5 4 ... 2 5 5]

 Accuracy: 
 63.53162473548084

 Precision of event Happening: 
 91.04477611940298

 Recall of event Happening: 
 100.0

 F-Score:
 0.5977668680507742


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [9 6 4 ... 2 3 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9811554179730846
Adjusted R-Squared:  0.9809583738644382
RMSE:  0.1971520990325714


Running Voting Regressor
R-Squared Value:  0.9940497595995743
Adjusted R-Squared:  0.9939875422569843
RMSE:  0.1107835829217884


-----
-----
-----
-----


##L1

In [22]:
# L1
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

def L1(df, label_col, nestimators, multi=False):
  df_cpy = df.copy()

  X,y = XYsplit(df_cpy, label_col)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

  scaler = StandardScaler()
  scaler.fit(X_train.fillna(0))

  sel_ = SelectFromModel((RandomForestClassifier(n_estimators = nestimators)))
  sel_.fit(scaler.transform(X_train.fillna(0)), y_train)
  selected_feat = X_train.columns[(sel_.get_support())]

  X_train_selected = sel_.transform(X_train.fillna(0))
  X_test_selected = sel_.transform(X_test.fillna(0))
  X_train_selected.shape, X_test_selected.shape

  ndf = df_cpy[df_cpy.columns.intersection(selected_feat.values)]
  ndf[label_col] = df_cpy[label_col]

  X,y = XYsplit(ndf.copy(), label_col)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

  RunClassificationAlgorithms(X_train, X_test, y_train, y_test, verbose=True, multi=multi)
  print('\n\n')
  RunRegressionAlgorithms(X_train, X_test, y_train, y_test)

###Running L1 with 25 estimators

In [23]:
print('Running on dataset1')
L1(dataset1, label_dataset1, 25, multi=False)
print('-----')
print('Running on dataset2')
L1(dataset2, label_dataset2, 25, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
L1(dataset3, label_dataset3, 25, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
L1(dataset4, label_dataset4, 25, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
L1(dataset5, label_dataset5, 25, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [0 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 1 1]

 Accuracy: 
 85.52631578947368

 Precision of event Happening: 
 87.70491803278688

 Recall of event Happening: 
 93.85964912280701

 AUC: 
 0.7719298245614036

 Confusion Matrix: 
 [[ 23  15]
 [  7 107]]

 F-Score:
 0.9067796610169491


Running LR
Prediction Vector: 
 [1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1
 1 1 1 1]

 Accuracy: 
 75.6578947368421

 Preci

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]

 Accuracy: 
 76.31578947368422

 Precision of event Happening: 
 76.35135135135135

 Recall of event Happening: 
 99.12280701754386

 AUC: 
 0.5350877192982456

 Confusion Matrix: 
 [[  3  35]
 [  1 113]]

 F-Score:
 0.8625954198473282


Running Random Forest
Prediction Vector: 
 [0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 85.52631578947368

 Precision of event Happening: 
 87.09677419

...

Prediction Vector: 
 [3 5 3 ... 3 5 5]

 Accuracy: 
 37.85563131906889

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.2449103598521767


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [7 5 3 ... 2 4 6]

 Accuracy: 
 65.69480366799905

 Precision of event Happening: 
 92.42424242424242

 Recall of event Happening: 
 100.0

 F-Score:
 0.646658309155232


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [9 6 4 ... 2 3 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9540914083390023
Adjusted R-Squared:  0.9539288808726547
RMSE:  0.3077195833273474


Running Voting Regressor
R-Squared Value:  0.9926778997642013
Adjusted R-Squared:  0.9926519777666707
RMSE:  0.12289267159805733


-----
-----
-----
-----


###Running L1 with 50 estimators

In [24]:
print('Running on dataset1')
L1(dataset1, label_dataset1, 50, multi=False)
print('-----')
print('Running on dataset2')
L1(dataset2, label_dataset2, 50, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
L1(dataset3, label_dataset3, 50, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
L1(dataset4, label_dataset4, 50, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
L1(dataset5, label_dataset5, 50, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 0 1]

 Accuracy: 
 82.23684210526315

 Precision of event Happening: 
 85.9504132231405

 Recall of event Happening: 
 91.22807017543859

 AUC: 
 0.7324561403508771

 Confusion Matrix: 
 [[ 21  17]
 [ 10 104]]

 F-Score:
 0.8851063829787233


Running LR
Prediction Vector: 
 [1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1
 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 1 1]

 Accuracy: 
 75.6578947368421

 Precis

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1]

 Accuracy: 
 77.63157894736842

 Precision of event Happening: 
 77.3972602739726

 Recall of event Happening: 
 99.12280701754386

 AUC: 
 0.5614035087719298

 Confusion Matrix: 
 [[  5  33]
 [  1 113]]

 F-Score:
 0.8692307692307693


Running Random Forest
Prediction Vector: 
 [0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 84.86842105263158

 Precision of event Happening: 
 86.4

 Recal

...

Prediction Vector: 
 [6 5 3 ... 2 7 5]

 Accuracy: 
 55.960498471667066

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.5015333633857311


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [7 6 3 ... 2 4 6]

 Accuracy: 
 66.09452151422526

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.6646187267663749


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [9 6 4 ... 2 3 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9525484020669557
Adjusted R-Squared:  0.9523916483220141
RMSE:  0.31284813590268984


Running Voting Regressor
R-Squared Value:  0.9960977963734611
Adjusted R-Squared:  0.9960849056583191
RMSE:  0.08971459785456619


-----
-----
-----
-----


###Running L1 with 100 estimators

In [25]:
print('Running on dataset1')
L1(dataset1, label_dataset1, 100, multi=False)
print('-----')
print('Running on dataset2')
L1(dataset2, label_dataset2, 100, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
L1(dataset3, label_dataset3, 100, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
L1(dataset4, label_dataset4, 100, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
L1(dataset5, label_dataset5, 100, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 0 1]

 Accuracy: 
 82.23684210526315

 Precision of event Happening: 
 85.36585365853658

 Recall of event Happening: 
 92.10526315789474

 AUC: 
 0.7236842105263158

 Confusion Matrix: 
 [[ 20  18]
 [  9 105]]

 F-Score:
 0.8860759493670887


Running LR
Prediction Vector: 
 [1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1
 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1
 1 1 1 1]

 Accuracy: 
 80.26315789473685

 Prec

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1]

 Accuracy: 
 77.63157894736842

 Precision of event Happening: 
 77.3972602739726

 Recall of event Happening: 
 99.12280701754386

 AUC: 
 0.5614035087719298

 Confusion Matrix: 
 [[  5  33]
 [  1 113]]

 F-Score:
 0.8692307692307693


Running Random Forest
Prediction Vector: 
 [0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 84.21052631578947

 Precision of event Happening: 
 86.290322580

...

Prediction Vector: 
 [6 5 3 ... 2 7 5]

 Accuracy: 
 55.960498471667066

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.5015333633857311


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [7 6 3 ... 2 4 6]

 Accuracy: 
 66.09452151422526

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.6646187267663749


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [9 6 4 ... 2 3 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9525484020669557
Adjusted R-Squared:  0.9523916483220141
RMSE:  0.31284813590268984


Running Voting Regressor
R-Squared Value:  0.9960974673291897
Adjusted R-Squared:  0.9960845755270681
RMSE:  0.0897183802625382


-----
-----
-----
-----


##RFE

In [26]:
# RFE (Recursive Feature Estimator)

def RFEImpl(df, label_col, multi=False):
  df_cpy = df.copy()

  X, y = traintestsplit(df_cpy, label_col)
  clf = RandomForestClassifier()
  selector = RFE(estimator=clf, step=1)
  # selector = RFECV(estimator=rfc, step=1, cv=StratifiedKFold(10), scoring='accuracy')
  selector = selector.fit(X,y)
  feature_list = X.columns[selector.support_].tolist()

  ndf = df_cpy[df_cpy.columns.intersection(feature_list)]
  ndf[label_col] = df_cpy[label_col]

  X,y = XYsplit(ndf.copy(), label_col)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

  RunClassificationAlgorithms(X_train, X_test, y_train, y_test, verbose=True, multi=multi)
  print('\n\n')
  RunRegressionAlgorithms(X_train, X_test, y_train, y_test)

In [27]:
print('Running on dataset1')
RFEImpl(dataset1, label_dataset1, multi=False)
print('-----')
print('Running on dataset2')
RFEImpl(dataset2, label_dataset2, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
RFEImpl(dataset3, label_dataset3, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
RFEImpl(dataset4, label_dataset4, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
RFEImpl(dataset5, label_dataset5, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 0 1]

 Accuracy: 
 83.55263157894737

 Precision of event Happening: 
 86.1788617886179

 Recall of event Happening: 
 92.98245614035088

 AUC: 
 0.7412280701754386

 Confusion Matrix: 
 [[ 21  17]
 [  8 106]]

 F-Score:
 0.8945147679324894


Running LR
Prediction Vector: 
 [0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1
 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 80.92105263157895

 Preci

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1]

 Accuracy: 
 76.97368421052632

 Precision of event Happening: 
 76.87074829931973

 Recall of event Happening: 
 99.12280701754386

 AUC: 
 0.5482456140350878

 Confusion Matrix: 
 [[  4  34]
 [  1 113]]

 F-Score:
 0.8659003831417624


Running Random Forest
Prediction Vector: 
 [0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 85.52631578947368

 Precision of event Happening: 
 86.50793650

...

Prediction Vector: 
 [6 5 4 ... 2 4 5]

 Accuracy: 
 54.33811427227839

 Precision of event Happening: 
 50.0

 Recall of event Happening: 
 11.475409836065573

 F-Score:
 0.5058070318262838


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [7 5 3 ... 2 4 5]

 Accuracy: 
 58.00611333176581

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.5519525207872491


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [9 6 4 ... 2 3 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9840536887835358
Adjusted R-Squared:  0.9839022518299132
RMSE:  0.1813586475794066


Running Voting Regressor
R-Squared Value:  0.9938572796434351
Adjusted R-Squared:  0.9937989442174467
RMSE:  0.1125611504020774


-----
-----
-----
-----


##Random Forest

In [28]:
#Random Forest

def RFImpl(df, label_col, nestimators, multi=False):
  df_cpy = df.copy()

  X, y = XYsplit(df_cpy, label_col)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

  scaler = StandardScaler()
  scaler.fit(X_train.fillna(0))

  clf = RandomForestClassifier(n_estimators = nestimators)
  sel_ = SelectFromModel(clf)
  sel_.fit(scaler.transform(X_train.fillna(0)), y_train)

  selected_feat = X_train.columns[(sel_.get_support())]

  ndf = df_cpy[df_cpy.columns.intersection(selected_feat.values)]
  ndf[label_col] = df_cpy[label_col]

  X,y = XYsplit(ndf.copy(), label_col)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

  RunClassificationAlgorithms(X_train, X_test, y_train, y_test, verbose=True, multi=multi)
  print('\n\n')
  RunRegressionAlgorithms(X_train, X_test, y_train, y_test)

###Running RF with 25 estimators

In [29]:
print('Running on dataset1')
RFImpl(dataset1, label_dataset1, 25, multi=False)
print('-----')
print('Running on dataset2')
RFImpl(dataset2, label_dataset2, 25, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
RFImpl(dataset3, label_dataset3, 25, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
RFImpl(dataset4, label_dataset4, 25, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
RFImpl(dataset5, label_dataset5, 25, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 0 1]

 Accuracy: 
 82.23684210526315

 Precision of event Happening: 
 84.8

 Recall of event Happening: 
 92.98245614035088

 AUC: 
 0.7149122807017544

 Confusion Matrix: 
 [[ 19  19]
 [  8 106]]

 F-Score:
 0.8870292887029289


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1
 1 1 1 1]

 Accuracy: 
 78.94736842105263

 Precision of even

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1]

 Accuracy: 
 77.63157894736842

 Precision of event Happening: 
 77.3972602739726

 Recall of event Happening: 
 99.12280701754386

 AUC: 
 0.5614035087719298

 Confusion Matrix: 
 [[  5  33]
 [  1 113]]

 F-Score:
 0.8692307692307693


Running Random Forest
Prediction Vector: 
 [0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 86.8421052631579

 Precision of event Happening: 
 87.9032258064

...

Prediction Vector: 
 [7 6 5 ... 2 3 6]

 Accuracy: 
 71.80813543381143

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.7040379130022956


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [6 6 4 ... 2 4 6]

 Accuracy: 
 78.86197977897955

 Precision of event Happening: 
 68.53932584269663

 Recall of event Happening: 
 100.0

 F-Score:
 0.7703029090942005


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [9 6 4 ... 2 3 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9659959203010423
Adjusted R-Squared:  0.9658755376728893
RMSE:  0.2648339532149425


Running Voting Regressor
R-Squared Value:  0.9967947042842414
Adjusted R-Squared:  0.9967833567657765
RMSE:  0.08130967069632011


-----
-----
-----
-----


###Running RF with 50 estimators

In [30]:
print('Running on dataset1')
RFImpl(dataset1, label_dataset1, 50, multi=False)
print('-----')
print('Running on dataset2')
RFImpl(dataset2, label_dataset2, 50, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
RFImpl(dataset3, label_dataset3, 50, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
RFImpl(dataset4, label_dataset4, 50, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
RFImpl(dataset5, label_dataset5, 50, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [0 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 1 1]

 Accuracy: 
 84.86842105263158

 Precision of event Happening: 
 87.60330578512396

 Recall of event Happening: 
 92.98245614035088

 AUC: 
 0.7675438596491228

 Confusion Matrix: 
 [[ 23  15]
 [  8 106]]

 F-Score:
 0.902127659574468


Running LR
Prediction Vector: 
 [0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1
 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 81.57894736842105

 Preci

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]

 Accuracy: 
 76.31578947368422

 Precision of event Happening: 
 76.35135135135135

 Recall of event Happening: 
 99.12280701754386

 AUC: 
 0.5350877192982456

 Confusion Matrix: 
 [[  3  35]
 [  1 113]]

 F-Score:
 0.8625954198473282


Running Random Forest
Prediction Vector: 
 [0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 86.8421052631579

 Precision of event Happening: 
 88.524590163

...

Prediction Vector: 
 [6 5 3 ... 2 7 5]

 Accuracy: 
 55.960498471667066

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.5015333633857311


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [7 6 3 ... 2 4 6]

 Accuracy: 
 66.09452151422526

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.6646187267663749


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [9 6 4 ... 2 3 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9525484020669557
Adjusted R-Squared:  0.9523916483220141
RMSE:  0.31284813590268984


Running Voting Regressor
R-Squared Value:  0.9960973062492998
Adjusted R-Squared:  0.9960844139150596
RMSE:  0.08972023183930915


-----
-----
-----
-----


###Running RF with 100 estimators

In [31]:
print('Running on dataset1')
RFImpl(dataset1, label_dataset1, 100, multi=False)
print('-----')
print('Running on dataset2')
RFImpl(dataset2, label_dataset2, 100, multi=False)
print('-----')
print('-----')
print('Running on dataset3')
RFImpl(dataset3, label_dataset3, 100, multi=True)
print('-----')
print('-----')
print('-----')
print('Running on dataset4')
RFImpl(dataset4, label_dataset4, 100, multi=False)
print('-----')
print('-----')
print('-----')
print('Running on dataset5')
RFImpl(dataset5, label_dataset5, 100, multi=True)
print('-----')
print('-----')
print('-----')
print('-----')

Running on dataset1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running Classification Algorithms
Running XgBoost
Prediction Vector: 
 [0 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 0 1]

 Accuracy: 
 84.21052631578947

 Precision of event Happening: 
 87.5

 Recall of event Happening: 
 92.10526315789474

 AUC: 
 0.7631578947368421

 Confusion Matrix: 
 [[ 23  15]
 [  9 105]]

 F-Score:
 0.8974358974358975


Running LR
Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1]

 Accuracy: 
 76.97368421052632

 Precision of even

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Prediction Vector: 
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1]

 Accuracy: 
 77.63157894736842

 Precision of event Happening: 
 77.3972602739726

 Recall of event Happening: 
 99.12280701754386

 AUC: 
 0.5614035087719298

 Confusion Matrix: 
 [[  5  33]
 [  1 113]]

 F-Score:
 0.8692307692307693


Running Random Forest
Prediction Vector: 
 [0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1]

 Accuracy: 
 87.5

 Precision of event Happening: 
 88.6178861788618

 Recall

...

Prediction Vector: 
 [4 5 3 ... 2 5 5]

 Accuracy: 
 45.074065365624264

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.37260801590617393


Running SVM


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [7 5 3 ... 3 4 5]

 Accuracy: 
 43.24006583588055

 Precision of event Happening: 
 0.0

 Recall of event Happening: 
 0.0

 F-Score:
 0.35081541055373494


Running Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction Vector: 
 [9 6 4 ... 2 3 6]

 Accuracy: 
 100.0

 Precision of event Happening: 
 100.0

 Recall of event Happening: 
 100.0

 F-Score:
 1.0





Running Regression Algorithms
Running Linear Regression
R-Squared Value:  0.9553086198818425
Adjusted R-Squared:  0.9551398139135019
RMSE:  0.30361276980676033


Running Voting Regressor
R-Squared Value:  0.984887813020904
Adjusted R-Squared:  0.9848307320502558
RMSE:  0.1765516618382713


-----
-----
-----
-----
